In [1]:
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.io import ImageReadMode, read_image
from torch import nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob

forestDirectory = '../../pics/2750/River'
forest_images = glob.glob(forestDirectory + '/*.jpg')

industrialDirectory = '../../pics/2750/Industrial'
industrial_images = glob.glob(industrialDirectory + '/*.jpg')

print(forest_images)
print(industrial_images)

industrial_labels = [0] * len(industrial_images)  # label 0 for industrial images
forest_labels = [1] * len(forest_images)  # label 1 for forest images

X = industrial_images + forest_images
Y = industrial_labels + forest_labels

class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image_path = self.images[index]
        label = self.labels[index]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Define the transformations
transform = T.Compose([
    T.ToTensor(),
    T.Resize((32, 32)),  # Resize the images to a consistent size
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image tensors
])

# Create the dataset
dataset = ImageDataset(X, Y, transform=transform)

# Define the train-validation split ratio (e.g., 0.8 for 80% training, 0.2 for 20% validation)
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset into train and validation sets
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create dataloaders for train and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.drop1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(120, 84)
        self.drop2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(84, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #x = x.view(x.size(0), -1)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)    

# Initialize the CNN model
model = CNN()

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Set the number of epochs
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_dataloader)
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")


['../../pics/2750/River\\River_1.jpg', '../../pics/2750/River\\River_10.jpg', '../../pics/2750/River\\River_100.jpg', '../../pics/2750/River\\River_1000.jpg', '../../pics/2750/River\\River_1001.jpg', '../../pics/2750/River\\River_1002.jpg', '../../pics/2750/River\\River_1003.jpg', '../../pics/2750/River\\River_1004.jpg', '../../pics/2750/River\\River_1005.jpg', '../../pics/2750/River\\River_1006.jpg', '../../pics/2750/River\\River_1007.jpg', '../../pics/2750/River\\River_1008.jpg', '../../pics/2750/River\\River_1009.jpg', '../../pics/2750/River\\River_101.jpg', '../../pics/2750/River\\River_1010.jpg', '../../pics/2750/River\\River_1011.jpg', '../../pics/2750/River\\River_1012.jpg', '../../pics/2750/River\\River_1013.jpg', '../../pics/2750/River\\River_1014.jpg', '../../pics/2750/River\\River_1015.jpg', '../../pics/2750/River\\River_1016.jpg', '../../pics/2750/River\\River_1017.jpg', '../../pics/2750/River\\River_1018.jpg', '../../pics/2750/River\\River_1019.jpg', '../../pics/2750/River

c:\Users\jiyoo\workspace\MakeAIWork2\env\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch [10/100], Loss: 0.0575, Accuracy: 0.9760
Epoch [20/100], Loss: 0.0347, Accuracy: 0.9880
Epoch [30/100], Loss: 0.0282, Accuracy: 0.9870
Epoch [40/100], Loss: 0.0318, Accuracy: 0.9880
Epoch [50/100], Loss: 0.0186, Accuracy: 0.9900
Epoch [60/100], Loss: 0.0170, Accuracy: 0.9890
Epoch [70/100], Loss: 0.0162, Accuracy: 0.9900
Epoch [80/100], Loss: 0.0182, Accuracy: 0.9910
Epoch [90/100], Loss: 0.0167, Accuracy: 0.9900
Epoch [100/100], Loss: 0.0149, Accuracy: 0.9910


Epoch [100/100], Loss: 0.0130, Accuracy: 0.9910
